In [46]:
import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline

In [47]:
movie=pd.read_csv('movies_metadata_modified.csv')

In [48]:
movie=movie[['id','original_title','original_language','adult']]
movie.head(2)

,id,original_title,original_language,adult
0,2,Ariel,fi,False
1,3,Varjoja paratiisissa,fi,False


In [50]:
movie.iloc[3000:9000]

,id,original_title,original_language,adult
3000,8867,The Punisher,en,False
3001,8868,Astérix et la surprise de César,fr,False
3002,8869,Eight Legged Freaks,en,False
3003,8870,Red Planet,en,False
3004,8871,How the Grinch Stole Christmas,en,False
3005,8872,Wayne's World,en,False
3006,8873,Wayne's World 2,en,False
3007,8874,My Best Friend's Wedding,en,False
3008,8875,Timber Falls,en,False
3009,8879,Pale Rider,en,False


In [5]:
ratings=pd.read_csv('ratings_small.csv',error_bad_lines=False,encoding='latin-1')

In [6]:
ratings.rename(columns={'movieId':'id'},inplace=True)

In [7]:
count_user=ratings['userId'].value_counts()
len(count_user)

671

In [8]:
rating=ratings[ratings['userId'].isin(count_user[count_user>100].index)]

In [9]:
len(rating)

80487

In [10]:
movie_with_rating=rating.merge(movie,on='id')

In [11]:
movie_with_rating.shape

(34640, 7)

In [12]:
movie_num_ratings=movie_with_rating.groupby('original_title')['rating'].agg('count').sort_values(ascending=False).reset_index().rename(columns={'rating':'num_rating'})

In [13]:
movie_with_rating_numrating=movie_with_rating.merge(movie_num_ratings,on='original_title')

In [14]:
min_num_rating=movie_with_rating_numrating[movie_with_rating_numrating['num_rating']>1000]

In [15]:
movie_with_rating_numrating.shape

(34640, 8)

In [16]:
movie_with_rating_numrating.drop_duplicates(['userId','original_title'],inplace=True)

In [17]:
len(movie_with_rating_numrating)#my srk has new name movie_with_rating_numrating

34496

In [18]:
pivot_table=movie_with_rating_numrating.pivot_table(index='original_title',columns='userId',values='rating').fillna(0)

In [19]:
pivot_table.shape

(2727, 258)

In [20]:
from scipy.sparse import csr_matrix
csr=csr_matrix(pivot_table)

In [21]:
type(csr)

scipy.sparse.csr.csr_matrix

In [22]:
from sklearn.neighbors import NearestNeighbors
nn=NearestNeighbors(metric='cosine',algorithm='brute')

In [23]:
nn.fit(csr)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [25]:
distances,movie=nn.kneighbors(pivot_table.iloc[0,:].values.reshape(1,258), n_neighbors=6)

In [26]:
movie=movie.ravel()
movie

array([   0, 1636, 2063, 1788,  998, 1216], dtype=int64)

In [27]:
for i in movie:
    print(pivot_table.index[i])

!Women Art Revolution
Premium Rush
The Eleventh Victim
Shadow Run
In the Name of the Father
Le fil


In [28]:
pivot_table.index=='Jarhead'

array([False, False, False, ..., False, False, False])

In [29]:
np.where(pivot_table.index=='Jarhead')

(array([1036], dtype=int64),)

In [30]:
np.where(pivot_table.index=='Jarhead')[0][0]

1036

In [35]:
def suggest(name):
    movie_id=np.where(pivot_table.index==name)[0][0]
    distance,movie=nn.kneighbors(pivot_table.iloc[movie_id,:].values.reshape(1,258),n_neighbors=10)
    for i in movie.ravel():
        print(pivot_table.index[i])

In [54]:
name=input('Enter a movie name')
suggest(name)

Enter a movie nameJarhead
Jarhead
Men in Black II
La science des rêves
Sleepless in Seattle
Edward Scissorhands
Los Olvidados
Terminator 3: Rise of the Machines
Lonely Hearts
座頭市
The Million Dollar Hotel
